tf2 keras 框架下实现  
参考  
  https://tensorflow.google.cn/guide/keras/train_and_evaluate#custom_metrics  
  https://tensorflow.google.cn/guide/keras/custom_callback  
  https://spaces.ac.cn/archives/4493  

两种loss实现的同时也对比了在keras自带Adam 和自定义AdaFactor 两种优化器下是否都可以执行

In [1]:
! pip install git+https://www.github.com/bojone/bert4keras.git
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.__version__)
tf.test.gpu_device_name()
device_lib.list_local_devices()
!nvidia-smi

  Cloning https://www.github.com/bojone/bert4keras.git to /tmp/pip-req-build-jabazvj0
  Running command git clone -q https://www.github.com/bojone/bert4keras.git /tmp/pip-req-build-jabazvj0
  Created wheel for bert4keras: filename=bert4keras-0.8.3-cp36-none-any.whl size=41795 sha256=1896511b84e7c4178eb9cf76b8b6ebc06e90fdc29564918e14536ddf8d3aff36
  Stored in directory: /tmp/pip-ephem-wheel-cache-4thsji5l/wheels/12/58/83/8ff5c864b80c860e6d9e9e0d90c04fafca05d01d21f9f6fcba
Successfully built bert4keras
2.2.0
Tue Jul 21 04:39:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      | 

## 标准输入loss_fun(y_true,y_pred)

In [4]:
from keras.layers import Input,Embedding,LSTM,Dense
from keras.models import Model
from keras import backend as K
from bert4keras.optimizers import *
import numpy as np

word_size = 128
nb_features = 10000
nb_classes = 10
encode_size = 64
# 构造模型
input = Input(shape=(None,))
embedded = Embedding(nb_features,word_size)(input)
encoder = LSTM(encode_size)(embedded)
predict = Dense(nb_classes, activation='softmax')(encoder)

def mycrossentropy(y_true, y_pred, e=0.1):
    loss1 = K.categorical_crossentropy(y_true, y_pred)
    loss2 = K.categorical_crossentropy(K.ones_like(y_pred)/nb_classes, y_pred)
    return (1-e)*loss1 + e*loss2

model = Model(inputs=input, outputs=predict)
model.compile(optimizer='adam', loss= mycrossentropy)

# 构造数据
data_size = 20
x_len = 10
x_array = np.random.randint(0,10,x_len*data_size).reshape((data_size,x_len))
#print(x_array)
y_len = nb_classes
y_array = [[0]*y_len for _ in list(range(data_size))]
y_ramdom = np.random.randint(0,y_len,data_size)
#print(y_ramdom)
for i,ind in enumerate(y_ramdom):
  y_array[i][ind]=1
y_array = np.array(y_array)
#print(y_array)
print(x_array.shape,y_array.shape)
# train
model.fit(x_array,y_array,epochs=1)

(20, 10) (20, 10)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
20/20 [==============================] - 0s 18ms/step - loss: 2.3068


### 测试自定义Optimizer AdaFactor
说明此处自定义optimizer 采用bert4keras 作者封装的AdaFactor用于测试
后续会单独拆解如何自定义optimizer

In [5]:
from bert4keras.optimizers import *
model2 = Model(inputs=input, outputs=predict)
model2.compile(optimizer=AdaFactor(), loss= mycrossentropy)
model2.fit(x_array,y_array,epochs=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
20/20 [==============================] - 0s 19ms/step - loss: 2.2981


## 自定义输入triplet loss

In [8]:
from keras.layers import Input,Embedding,LSTM,Dense,Lambda
from keras.layers.merge import dot
from keras.models import Model
from keras import backend as K

word_size = 128
nb_features = 10000
nb_classes = 10
encode_size = 64
margin = 0.1
# =============================================================================
## 模型构造
embedding = Embedding(nb_features,word_size)
lstm_encoder = LSTM(encode_size)

def encode(input):
    return lstm_encoder(embedding(input))

q_input = Input(shape=(None,))
a_right = Input(shape=(None,))
a_wrong = Input(shape=(None,))
q_encoded = encode(q_input)
a_right_encoded = encode(a_right)
a_wrong_encoded = encode(a_wrong)

q_encoded = Dense(encode_size)(q_encoded) #一般的做法是，直接讲问题和答案用同样的方法encode成向量后直接匹配，但我认为这是不合理的，我认为至少经过某个变换。

right_cos = dot([q_encoded,a_right_encoded], -1, normalize=True)
wrong_cos = dot([q_encoded,a_wrong_encoded], -1, normalize=True)

loss = Lambda(lambda x: K.relu(margin+x[0]-x[1]))([wrong_cos,right_cos])

model_train = Model(inputs=[q_input,a_right,a_wrong], outputs=loss)
model_q_encoder = Model(inputs=q_input, outputs=q_encoded)
model_a_encoder = Model(inputs=a_right, outputs=a_right_encoded)

model_train.compile(optimizer='adam', loss=lambda y_true,y_pred: y_pred)
model_q_encoder.compile(optimizer='adam', loss='mse')
model_a_encoder.compile(optimizer='adam', loss='mse')
# =============================================================================
## 数据构造
# q : id序列，a1 a2 与 q 一样尺寸的 0/1 位置标记
data_size = 20
x_len = 10
q = np.random.randint(0,10,x_len*data_size).reshape((data_size,x_len))
#print(q)

y_len = x_len
a1 = [[0]*y_len for _ in list(range(data_size))]
a2 = [[0]*y_len for _ in list(range(data_size))]
y_ramdom = np.random.randint(0,y_len,data_size)
y_ramdom = list(map(lambda x : x if x<9 else 0 ,y_ramdom))
y_wrong = [min(x+1,y_len-1) for x in y_ramdom]#负采样
#print(y_ramdom,y_wrong)

for i,ind in enumerate(y_ramdom):
  a1[i][ind]=1
for i,ind in enumerate(y_wrong):
  a2[i][ind]=1
a1 = np.array(a1)
a2 = np.array(a2)
#print(a1,a2)
print(q.shape,a1.shape)
y = np.random.randint(0,2,1*data_size).reshape((data_size,1))
#print(y,y.shape)
# =============================================================================
## train
model_train.fit([q,a1,a2], y, epochs=10)
#其中q,a1,a2分别是问题、正确答案、错误答案的batch，y是任意形状为(len(q),1)的矩阵

(20, 10) (20, 10)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
20/20 [==============================] - 1s 49ms/step - loss: 0.0961
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0778
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0622
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.0556
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0498
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0435
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0389
Epoch 8/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0350
Epoch 9/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0309
Epoch 10/10
20/20 [==============================] - 0s 4ms/step - loss: 0.0258


In [12]:

q_temp = np.random.randint(0,10,20).reshape((2,10))
print(model_q_encoder.predict(q_temp))
print(model_a_encoder.predict(q_temp))

[[ 0.01453993 -0.00562698  0.01424397 -0.00596067  0.02922352 -0.00973755
   0.0088687   0.00974386 -0.02046435  0.0235047  -0.03514804 -0.01871555
   0.00842636 -0.05295112 -0.0209622  -0.06124398  0.00663771 -0.01142377
  -0.03086904  0.03342791  0.01062222  0.02521044 -0.0237724  -0.00272057
  -0.02169065  0.01687902  0.02022937 -0.01795528 -0.00012353  0.04738297
  -0.04679681 -0.02535913 -0.04048171  0.00371677 -0.02239585  0.03536025
   0.04912126  0.01894899 -0.00855447  0.01653391 -0.00767307  0.00935317
  -0.01510719  0.00748171  0.00387116  0.02085221  0.03900766  0.04929505
   0.00652571 -0.02875523  0.00613667  0.01378987  0.01051747 -0.02158486
  -0.03003236  0.01224885 -0.06878784 -0.02980276  0.01651664 -0.03540721
   0.00279418 -0.01783215 -0.0127061   0.00735367]
 [ 0.0040168  -0.01170977  0.00683147  0.00996666  0.03682316 -0.02048496
  -0.00367504  0.00371774 -0.01962885  0.01289939 -0.02837418 -0.01111015
   0.01568361 -0.06167425 -0.0020009  -0.0394701   0.01570628

### 测试自定义Optimizer AdaFactor

In [13]:
model_train2 = Model(inputs=[q_input,a_right,a_wrong], outputs=loss)
model_q_encoder2 = Model(inputs=q_input, outputs=q_encoded)
model_a_encoder2 = Model(inputs=a_right, outputs=a_right_encoded)

model_train2.compile(optimizer=AdaFactor(1e-3), loss=lambda y_true,y_pred: y_pred)
model_q_encoder2.compile(optimizer=AdaFactor(1e-3), loss='mse')
model_a_encoder2.compile(optimizer=AdaFactor(1e-3), loss='mse')

model_train2.fit([q,a1,a2], y, epochs=10)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
20/20 [==============================] - 1s 50ms/step - loss: 0.0214
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.1119
Epoch 3/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1115
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.1111
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.1106
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.1101
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.1096
Epoch 8/10
20/20 [==============================] - 0s 4ms/step - loss: 0.1090
Epoch 9/10
20/20 [==============================] - 0s 4ms/step - loss: 0.1084
Epoch 10/10
20/20 [==============================] - 0s 4ms/step - loss: 0.1078


In [14]:
q_temp = np.random.randint(0,10,20).reshape((2,10))
print(model_q_encoder.predict(q_temp))
print(model_a_encoder.predict(q_temp))

[[ 0.17013536 -0.12803134  0.81835103 -0.40400583  0.5529031   0.34814867
  -0.12547715 -0.23404755  0.35967755 -0.5073197  -0.14004096  0.4665255
   0.7347514   0.31813377  0.03673103  0.08659136  0.09272325  0.28349522
   0.9962105  -0.07956529 -0.76867646 -0.41571182 -0.5606564   0.06912786
  -0.22771195  0.06609165 -0.02848083 -0.48501885 -0.03491921 -0.24105152
  -0.19094773  0.07582311 -0.14129163  0.19157688  0.5998938  -0.06636051
  -0.71505654 -0.05728829 -0.05894502  0.22055946  0.5279871  -0.21601668
  -0.59748983  0.15371923 -0.21674079 -0.07280705  0.05722732 -0.7410108
  -0.05459466 -0.42537442 -0.11368894 -0.46594083 -0.03504705 -0.00372955
   0.67588544  0.10958757 -0.18258744 -0.3903203  -0.07150177 -0.09541807
   0.17884365 -0.31851897 -0.22108732  0.51267284]
 [ 0.03355308  0.00575609  0.6730226  -0.13472328  0.43689182  0.25743672
  -0.2774217  -0.24944371  0.37511334 -0.51027733 -0.22842759  0.5848009
   0.7888117  -0.12492432  0.2691981   0.07849284  0.08013863 -0